## 패스트텍스트(FastText)
- https://wikidocs.net/22883
- 단어를 벡터로 만드는 또 다른 방법으로는 페이스북에서 개발한 FastText
- 메커니즘 자체는 Word2Vec의 확장
    - Word2Vec와 FastText와의 가장 큰 차이점
        - Word2Vec는 단어를 쪼개질 수 없는 단위로 생각한다면, FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주
        - 내부 단어. 즉, 서브워드(subword)를 고려하여 학습

### 내부 단어(subword)의 학습
- FastText에서는 각 단어는 글자 단위 n-gram의 구성으로 취급
- n을 몇으로 결정하는지에 따라서 단어들이 얼마나 분리되는지 결정
- 예) n=3, apple -> app, ppl, ple로 분리, 벡터 생성
    - <ap, app, ppl, ple, le> 
    - <apple (특별토큰)> 
    - n = 3인 경우, FastText는 단어 apple에 대해서 6개의 토큰을 벡터화
- 실제 사용할 때는 n의 최소값과 최대값으로 범위를 설정 가능
- 기본값으로는 각각 3과 6으로 되어 있음
- 단어 apple 에 대해 아래 내부 단어 벡터화 진행
    - 벡터화: 저 단어들에 대해서 Word2Vec을 수행
    - "<ap, app, ppl, ppl, le>, <app, appl, pple, ple>, <appl, pple>, ..., <apple>"

### 모르는 단어(Out Of Vocabulary, OOV)에 대한 대응
- 장점: (데이터셋 충분하다면) 위와 같은 내부 단어(Subword)를 통해 모르는 단어(Out Of Vocabulary, OOV)에 대해서도 다른 단어와의 유사도를 계산 가능
    - 예) birthplace 출생지란 단어 학습 x, birth, place 학습 -> FastText는 birthplace의 벡터 얻기 가능
    - 모르는 단어에 제대로 대처할 수 없는 Word2Vec, GloVe와는 다른 점

### 단어 집합 내 빈도 수가 적었던 단어(Rare Word)에 대한 대응
- Word2Vec의 경우에는 등장 빈도 수가 적은 단어(rare word)에 대해서는 임베딩의 정확도가 높지 않다는 단점
- FastText의 경우, 만약 단어가 희귀 단어라도, 그 단어의 n-gram이 다른 단어의 n-gram과 겹치는 경우라면, Word2Vec과 비교하여 비교적 높은 임베딩 벡터값
    - 노이즈가 많은 코퍼스에서 강점 (ex. 오타 appple)    

### 실습으로 비교하는 Word2Vec Vs. FastText

In [1]:
import pickle

In [2]:
with open("result.pickle","rb") as fi:
    result = pickle.load(fi) #result 불러오기

In [5]:
from gensim.models import FastText

model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [6]:
# size -> vector_size

In [7]:
model.wv.most_similar("electrofishing")

[('electrolux', 0.8671838641166687),
 ('electrolyte', 0.866204023361206),
 ('electro', 0.851755678653717),
 ('electroshock', 0.8460729718208313),
 ('electroencephalogram', 0.8371050953865051),
 ('electrochemical', 0.8337551951408386),
 ('electric', 0.8274891376495361),
 ('electronic', 0.8268327116966248),
 ('electrogram', 0.8268225789070129),
 ('electron', 0.8242252469062805)]

### 한국어에서의 FastText
- 한국어의 경우에도 OOV 문제를 해결하기 위해 FastText를 적용하고자 하는 시도
- 1. 음절단위
    - 예) n=3, '자연어처리'
    - <자연, 자연어, 연어처, 어처리, 처리>
- 2. 자모 단위
    - 더 나아가 자모 단위(초성, 중성, 종성 단위)로 임베딩하는 시도
    - 음절 단위가 아니라, 자모 단위로 가게 되면 오타나 노이즈 측면에서 더 강한 임베딩을 기대
    - 예) 분리된 결과 : ㅈ ㅏ _ ㅇ ㅕ ㄴ ㅇ ㅓ _ ㅊ ㅓ _ ㄹ ㅣ _
    - n=3, < ㅈ ㅏ, ㅈ ㅏ _, ㅏ _ ㅇ, ... 중략>